# Export to shapefile (and other useful formats)
MIKE IO 1D supports exporting to shapefile (and other geospatial features) via GeoPandas. To use these features, you will need to ensure GeoPandas and its dependencies are installed.

In [ ]:
from mikeio1d import Res1D

In [ ]:
res = Res1D("../tests/testdata/network.res1d")
res

## Read res1d file into a GeoDataFrame (static network only)

In [ ]:
# Let's read the network as a geopandas dataframe.
df = res.network.to_geopandas()
df.head()

In [ ]:
# Now let's plot the network.
df.plot()

## Read res1d into a GeoDataFrame (with quantities)

- To include quantities, you need to go a bit deeper into the network (res.reaches instead of res.network).
- Additionally, you need to specify how you would like to aggregate the results in time (and other dimensions like chainage)

In [ ]:
# Here we read the reaches, and choose to get the maximum value of all available quantities.
df_reaches = res.reaches.to_geopandas(agg='max')
df_reaches.head()

In [ ]:
# Similarly, we can do the same for nodes.
df_nodes = res.nodes.to_geopandas(agg='max')
df_nodes.head()

In [ ]:
# Now let's plot the max discharge for each reach.
df_reaches.plot(column='max_Discharge', cmap="RdYlGn_r", legend=True)

In [ ]:
# Similarly, we can do the same for nodes. Note we plot the reaches in the background for context.
ax = df_reaches.plot(color="gray", zorder=-1)
df_nodes.plot(ax=ax, column='max_WaterLevel', legend=True)


In [ ]:
# Finally, we look at a similar example but for catchments.
res = Res1D("../tests/testdata/catchments.res1d")
df_catchments = res.catchments.to_geopandas('max')
df_catchments.head()

In [ ]:
# Let's plot the catchment runoff.
df_catchments.plot(column='max_TotalRunOff', cmap='Blues', legend=True, alpha=0.75)

# Interactive plotting with explore

### Add an interactive plot directly in the notebook

Note: The map does not display on GitHub. You may use https://nbviewer.org/ to properly view the notebook.

In [ ]:
map = df_reaches.explore(column="max_Discharge", legend=True, tiles="cartodb positron", tooltip=["name", "max_Discharge"], popup=True)
map

### Save your interactive map into a shareable HTML file

In [ ]:
map.save("results.html")

## Export to shapefile

In [ ]:
# Shapefiles require all geometries to be of the same type
df_reaches.to_file("reaches.shp")
df_nodes.to_file("nodes.shp")
df_catchments.to_file("catchments.shp")

## Clean up

In [ ]:
from pathlib import Path

files_to_delete = []
for p in ("results.html", "reaches*", "nodes*", "catchments*"):
    files_to_delete = [*files_to_delete, *Path.glob(Path.cwd(), p)]

for f in files_to_delete:
    f.unlink()